To do:
* Change scraping to be off what's new page not RSS feed - doesn't seem to be working for e.g. Greenwich. Get date off the same page, as well as folder title
* Get meeting title from scraping meeting page - DONE
* Include support for docs as well as pdf - DONE
* Gather all text for one council for each week - DONE
* text has a lot of \n in it, which can stop multi-word searches - can replace these (but test to see how it works first (does it think it's text or special characters?), and test to see if replacement is needed to search in python or if it doesn't actually matter - find an example phrase e.g. 'a questionnaire was \n \napproved')
* move extracted folder to still be in 'files' folder for github purposes - DONE I THINK
* Delete all pdfs and docxs to save space
* Do general tidy up and commenting
* Turn into one python file that defines and runs extract_council_text, so then can have another file that checks for each person in DB and creates email
* Have list of all councils, but only get PDFs for ones that people have signed up for (i.e. check database first)
* at end, do file that archives all text files by grouping them into a dated folder in an archive folder in dropbox
* why does it sometimes stop loading? Is there scraping protection?


Notes on first runthrough:
* seems to work (yay)
* but len(output) is only 42 - for example output[40] has multiple in it
* think it has a separate item in the list for the text, pdf and docx. This would mean there are 42/13 = 14 meetings - correct
* solved by changing extract_text to be extend not append - TO TEST

In [1]:
import pandas as pd
import PyPDF2
from bs4 import BeautifulSoup
import urllib
import requests
import re
import simplejson
import os
import shutil
import random
import docx2txt

In [18]:
# get list of councils with name and url of RSS feed
# for now just take one as an example

council_name = 'Chester'
council_url = 'http://cmttpublic.cheshirewestandchester.gov.uk/mgWhatsNew.aspx?bcr=1'

In [3]:
def get_meetings(soup, base_url):
    doclist = soup.find('ul', 'mgDateList')
    meeting_list = []
    
    for li in doclist.find_all('li'):
        date = li.find('span').text
        href = li.find('a')
        url = base_url + href.get('href')
        title = href.text
        short_title = title[title.rfind(",")+2:]
        meeting_list.append([date, short_title, url_replace(base_url,li), url])
        
    return meeting_list

In [4]:
def url_replace(base_url, li):
    href = li.find('a')
    new_url = base_url + href.get('href')
    replacer = 'href = ' + new_url + '"'
    replaced = re.compile('href.*"')
    return re.sub(replaced, replacer, str(li))

In [5]:
escapes = [chr(char) for char in range(1, 32)]
def tidy2(s):
    for e in escapes:
        if e in s:
            s = s.replace(e,"")
    return s

In [6]:
def get_link(item):
    # gets link from bs4 item, as bs4 doesn't seem to be working
    item = str(item)
    link_start = item.find('<link/>')+len('<link/>')
    item_short = item[link_start:]
    link = item_short[:item_short.find('\r')]
    return link

In [7]:
escapes = [chr(char) for char in range(1, 32)]
def tidy(s):
    for e in escapes:
        if e in s:
            s = s[:s.find(e)]
    return s

In [8]:
def get_urls_old(soup):
    item_all = soup.find_all('item')
    url_list = []
    for item in item_all:
        url_list.append([item.find('title').text, get_link(item)])
    return url_list

In [9]:
def directory(base, path):
    directory = base + path
    if os.path.exists(directory):
        while os.path.exists(directory):
            path = path + str(random.randint(1,100))
            directory = base + path
        os.makedirs(directory)
    else:
        os.makedirs(directory)
    return directory + '/', path

In [10]:
def meeting_scrape(url):
    # get correct part of page
    url_get = requests.get(url)
    item_soup = BeautifulSoup(url_get.content, 'lxml')
    modgov = item_soup.find('div', id = 'modgov')
    item_text = modgov.text

    pdf_link_list = modgov.find_all(href = re.compile(".pdf"))
    pdf_list = []
    for link in pdf_link_list:
        pdf_title = tidy(link.text).replace('/','-').replace(':','-')[:100].rstrip() + '.pdf'
        pdf_list.append([link['href'],pdf_title])
        
    doc_link_list = modgov.find_all(href = re.compile(".docx"))
    doc_list = []
    for link in doc_link_list:
        doc_title = tidy(link.text).replace('/','-').replace(':','-')[:100].rstrip() + '.docx'
        doc_list.append([link['href'],doc_title])

    return item_text, pdf_list, doc_list

In [11]:
def download_file(file_list, base_url, meeting_dir):
    # download pdfs from list
    for file_url, file_title in file_list:
        full_file_url = base_url + file_url
        response = requests.get(full_file_url)
        with open(meeting_dir + file_title, 'wb') as f:
            f.write(response.content)
            print ('Added: ' + file_title)

In [12]:
def extract_pdf (pdf_list, directory, meeting_title, date):
    
    all_text = []
    
    for url, pdf_title in pdf_list:
        pdfFileObj = open(directory + pdf_title, 'rb') 

        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

        length = pdfReader.numPages

        pdf_text = ''

        for i in range(0,length): 
            pageObj = pdfReader.getPage(i)
            page_text = pageObj.extractText()
            pdf_text = pdf_text + ' ' + page_text

        all_text.append([meeting_title, date, pdf_title, pdf_text])

        pdfFileObj.close()
    
    return all_text

In [13]:
def extract_docx(docx_list, directory, meeting_title, date):
    
    all_text = []
    
    for url, docx_title in docx_list:
        docx_text = docx2txt.process(directory + docx_title)
        all_text.append([meeting_title, date, docx_title, docx_text])
    
    return all_text

In [34]:
def extract_file(pdf_list, docx_list, item_text, directory, folder, meeting_title, date):
    print('Extracting text for: ' + folder)
    
    txt_list = [[meeting_title, date,'page text', item_text]]
    
    txt_list.extend(extract_pdf(pdf_list, directory, meeting_title, date))
    
    txt_list.extend(extract_docx(docx_list, directory, meeting_title, date))
    
    f = open(directory + 'text.txt', 'w')
    simplejson.dump(txt_list, f)
    f.close()
    print ('Extracted text for: ' + folder)
    

In [39]:
def combine_text(council_name, meeting_dir_list):
    base = 'C:/Users/Jack/Documents/Software and Programming/Gov Alert/Files/'
    files = [base + location+'/text.txt' for location in meeting_dir_list]
    with open(council_name +'.txt','w') as of:
        output = []
        for f in files:
            with open(f,'r') as fc:
                output.extend(simplejson.load(fc))
        simplejson.dump(output, of)

In [16]:
def extract_council_text(council_name, council_url):
    
    base_url = council_url[:council_url.find('.gov.uk/')+8]
    
    # get list of urls from the latest week
    url_get = requests.get(council_url)
    soup = BeautifulSoup(url_get.content, 'lxml')
    meetings = get_meetings(soup, base_url)
    
    base_dir, council_folder = directory('C:/Users/Jack/Documents/Software and Programming/Gov Alert/Files/', council_name)
    print('Folder created: ' + council_folder)
    
    meeting_dir_list = []
    
    # get text file for each meeting
    for meeting in meetings:
        # create folder
        folder = meeting[1][:40].replace('/','-').replace(':','-').rstrip()
        meeting_dir, folder = directory(base_dir, folder)
        meeting_dir_list.append(meeting_dir)
        print('Folder created: ' + folder)
        
        # to be updated with better scraping
        meeting_title = meeting[1]
        date = meeting[0]
        
        # scrape page
        item_text, pdf_list, docx_list = meeting_scrape(meeting[3])
        
        # download all documents
        download_file(pdf_list + docx_list, base_url, meeting_dir)
        
        # extract text from all files
        extract_file(pdf_list, docx_list, item_text, meeting_dir, folder, meeting_title, date)
    
    # gather all text into one folder
    combine_text(council_name, meeting_dir_list)
    
    # delete all unnecessary files
    

In [19]:
extract_council_text(council_name, council_url)

Folder created: Chester
Folder created: Licensing Committee
Added: Agenda frontsheet.pdf
Added: Agenda reports pack.pdf
Added: Minutes  PDF 105 KB.pdf
Added: Temporary Event Notices DOCX 40 KB.docx
Added: Review of the Council's Statement of Licensing Policy DOCX 38 KB.docx
Added: App 1 LA03 Policy 2019 , item 6..docx
Added: App 2 LA03 Policy 2019 , item 6..docx
Extracting text for: Licensing Committee
Extracted text for: Licensing Committee
Folder created: Cheshire West and Chester Overview and S
Added: Agenda frontsheet.pdf
Added: Agenda reports pack.pdf
Added: Minutes  PDF 77 KB.pdf
Added: Cheshire West & Chester Overview & Scrutiny Committee Work Programme 2018-19  PDF 245 KB.pdf
Extracting text for: Cheshire West and Chester Overview and S
Extracted text for: Cheshire West and Chester Overview and S
Folder created: Health and Wellbeing Board
Added: Agenda frontsheet.pdf
Added: Additional Document - Focus on our Care Communities Presentation Slides  PDF 2 MB.pdf
Added: Minutes  PDF

Extracted text for: People Overview and Scrutiny Committee
Folder created: People Overview and Scrutiny Committee82
Added: Agenda frontsheet.pdf
Added: Additional Document - Annual Review  PDF 4 MB.pdf
Added: Additional Document - Winter Preparedness Presentation - Vale Royal CCG  PDF 755 KB.pdf
Added: Additional Document - West Cheshire CCG Presentation Slides  PDF 633 KB.pdf
Added: Minutes  PDF 75 KB.pdf
Added: Winter Plans Update  PDF 572 KB.pdf
Added: OSC presentation re surge and resilience planning V2pdf , item 6..pdf
Added: Vale Royal CCG Financial Recovery Plan Update  PDF 4 MB.pdf
Added: Review of Disabled Children's Services  PDF 330 KB.pdf
Added: Council's Safeguarding Duties- Recommendation 41 Visits  PDF 163 KB.pdf
Added: Rec 41 Childrens Dis Team July 2018 Cllr Armstrong , item 9..pdf
Added: Rec 41 Visit - Aug 2018 Cllr Tonge , item 9..pdf
Added: Rec 41  2018 Cllr M Parker , item 9..pdf
Added: Rec 41 Visit - Aug 2018 Cllr Houlbrook , item 9..pdf
Added: Information Only It

Extracted text for: Audit and Governance Committee
Folder created: District Advisory Panel - Northwich and
Added: Agenda frontsheet.pdf
Added: Agenda reports pack.pdf
Added: Minutes of the last meeting  PDF 57 KB.pdf
Added: Membership and Terms of Reference DOCX 16 KB.docx
Extracting text for: District Advisory Panel - Northwich and
Extracted text for: District Advisory Panel - Northwich and
Folder created: Health and Wellbeing Board64
Added: Agenda frontsheet.pdf
Added: Additional Document - Focus on our Care Communities Presentation Slides  PDF 2 MB.pdf
Added: Minutes  PDF 81 KB.pdf
Added: Focus on our Care Communities  PDF 2 MB.pdf
Added: Eat Well, Be Active Update  PDF 389 KB.pdf
Added: Review of the Better Care Fund  PDF 604 KB.pdf
Added: The Community Safety Partnership - An Overview  PDF 450 KB.pdf
Added: Seasonal Influenza Update and Winter Campaign  PDF 569 KB.pdf
Added: Children's Trust Update  PDF 358 KB.pdf
Added: Draft Agenda for the October Health and Wellbeing Board Meet

In [21]:
with open('chester.txt', 'r') as f:
    output = simplejson.load(f)

In [38]:
output[5]

[]